In [1]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve
from IPython.display import SVG, display
from scipy.constants import mu_0, epsilon_0

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 3: Problem 3 (p33)

## Physics Constants

In [2]:
K0 = 2 * np.pi * 3E8 * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0

## 3-noded Linear Triangular Elements, $P_1$

In [3]:
FINITE_ELEMENT = ("Triangle", 1)

BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': None, 'name': 'circular_scatterer'}]

MATERIAL = [{'tag': 201, 'name': 'free_space', 'relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 301, 'name': 'PML_a','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 302, 'name': 'PML_b','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 303, 'name': 'PML_c','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 304, 'name': 'PML_d','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 401, 'name': 'PML_I','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 402, 'name': 'PML_II','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 403, 'name': 'PML_III','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1},
            {'tag': 404, 'name': 'PML_IV','relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1}]

# Criar geometria coaxial
GEOMETRY = {'h': WAVELENGTH/25, 'L': WAVELENGTH*0.5, 'ra': WAVELENGTH*0.5, 'x0': 1.0*WAVELENGTH}
mesh_data = ps.create_p4_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, GEOMETRY, view_mesh=False)

Malha salva em pre_processing/mesh/rectangular_pml_domain_Triangle1.msh
Model rectangular_pml (2D)
Info     : 51 geometric entities
Info     : 10 Physical Groups
Info     : 6316 nodes in total
Info     : 12249 2 elements in total


## FEM Formulation

In [4]:
error_dict = {}
for order_n in [1.0, 2.0, 3.0]:
    # Parâmetros da PML
    PML_DESIGN = {'x0': GEOMETRY['x0'], 'n': order_n, 'R': 1E-4}
    
    # Aplicar as propriedades da PML
    mesh_data = ps.apply_pml_physics(FINITE_ELEMENT, PML_DESIGN, mesh_data)

    # Montagem das matrizes globais
    Sg, Mg, bg = ps.pml_global_matrices(FINITE_ELEMENT, mesh_data)

    # Aplicar condições de contorno ABC
    Sg, Mg, bg = ps.apply_simple_bcs(Sg, Mg, bg, mesh_data)

    # Solução do problema matricial
    uh = spsolve((Sg + Mg).tocsr(), bg.toarray())

    # Dicionário com os resultados numéricos
    ezh = {key+1: value for key, value in enumerate(uh)}

    # Domínio físico do problema
    physical_mesh_data = ps.get_physical_mesh_data(mesh_data)

    # Cálculo do erro
    summ = 0
    Nn = len(physical_mesh_data['nodes'])
    for key, node in physical_mesh_data['nodes'].items():
        ez_phy = ps.ez_at_node(node, GEOMETRY['ra'])
        ezh_phy = ezh[key]
        summ += (ez_phy - ezh_phy)**2
    error_dict[order_n] = np.abs(np.abs(np.sqrt(summ/Nn)))

print("Error dictionary:\n", error_dict)

Error dictionary:
 {1.0: 0.0014115713619540845, 2.0: 0.0030508819832137414, 3.0: 0.07782712776834017}


Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.